## The method of stock prediction with various analize method (For practice)
- This is my python project for improving my skill
- For stock prediction, the proposed method uses various algorithms such as statistical analysis, probability model and neural networks
- The simple s&p500 data is used


## 다양한 분석 기법을 활용한 주가 예측 방법 (연습) 
- 파이썬을 활용하여 다양한 분석 기법을 활용한 주가 예측 방법 구현 수행 (습작용)
- 그냥 널리 알려져 있는 통계적 분석, 확률모델, 인공 신경망 등등 여러 방법 활용
- 데이터 역시 단순한 종가 등 수학적 정보만 활용 
- 어떤 분석 방법이 예측력이 좋은지, 어떻게 하면 예측력을 높일 수 있는지 등

---
## 시도해볼것들 
### 토픽 모델링 
### 인공 신경망 
### 평가 방법 
### 비교 평가 
--- 
## Reference 
- http://3months.tistory.com/137 
    - 파이썬으로 선형 회귀 분석
- http://uncoded.tistory.com/21 
    - 파이썬으로 로지스틱 회귀 분석 구현 

    

In [3]:
''' 
Library  
'''
# default 
import numpy as np 
import pandas as pd

# For regression   
from sklearn import linear_model 
import matplotlib.pyplot as plt 



## 1) Data read and describe part 
- using pandas, read csv file 
---
- pandas 라이브러리를 활용하여 stock data(csv)를 읽어오자 
- 데이터에 대한 성격을 보자 
- 데이터 분리 622 / 82 


In [60]:
'''
date = 날짜 
open = 시가? - 시작 시 가격  
high = 고가 - 당일 최고가격 
low = 저가 - 당일 최저가격 
close = 종가 - 장 마감 가격 
volume = 거래량 
Name = sp500 회사이름 - 약어 
'''
temp_sp500_data = pd.read_csv('all_stocks_5yr.csv', header= 0, names=['date','open','high','low','close','volume','Name'])
temp_sp500_data.describe()

,open,high,low,close,volume
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08


In [104]:
sp500_data = temp_sp500_data
sp500_data = pd.concat([sp500_data, sp500_data['close'].shift(1).rename('lag1'), 
                        sp500_data['close'].shift(2).rename('lag2'), 
                        sp500_data['close'].shift(3).rename('lag3'),
                        sp500_data['close'].shift(4).rename('lag4'), 
                        sp500_data['close'].shift(5).rename('lag5'),
                        sp500_data['close'].shift(6).rename('lag6'), 
                        sp500_data['close'].shift(7).rename('lag7'), 
                        sp500_data['close'].shift(-1).rename('fu1'),
                       sp500_data['close'].shift(-7).rename('fu7'),
                       sp500_data['close'].shift(-30).rename('fu30')], 
                        axis=1).dropna()

In [105]:
sp500_data[:2]

,date,open,high,low,close,volume,Name,lag1,lag2,lag3,lag4,lag5,lag6,lag7,fu1,fu7,fu30
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL,14.26,14.50,13.99,14.66,14.27,14.46,14.75,13.37,13.61,15.69
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL,13.33,14.26,14.50,13.99,14.66,14.27,14.46,13.57,13.90,15.72


In [106]:
print('A number of company in sp500 data is ', len(list(set(sp500_data['Name']))) ) 

A number of company in sp500 data is  505


In [107]:
#split data 
print(len(sp500_data))
_622_data = sp500_data[0:int(len(sp500_data)*0.6)]
_622_data_eval = sp500_data[int(len(sp500_data)*0.6)+1:int(len(sp500_data)*0.8)]
_622_data_test = sp500_data[0:int(len(sp500_data)*0.8)+1:-1]

_82_data = sp500_data[0:int(len(sp500_data)*0.8)]
_82_data_eval = sp500_data[int(len(sp500_data)*0.8)+1:-1]




618992


In [108]:
_622_data[:3]

,date,open,high,low,close,volume,Name,lag1,lag2,lag3,lag4,lag5,lag6,lag7,fu1,fu7,fu30
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL,14.26,14.50,13.99,14.66,14.27,14.46,14.75,13.37,13.61,15.69
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL,13.33,14.26,14.50,13.99,14.66,14.27,14.46,13.57,13.90,15.72
9,2013-02-22,13.57,13.60,13.21,13.57,6071400,AAL,13.37,13.33,14.26,14.50,13.99,14.66,14.27,13.02,14.05,15.84


## 2) 회귀모델 적용(regression model) 
- 선형 회귀 모델을 이용한 지수 예측 
- 로지스틱 회귀 모델을 이용한 up/down 예측 
- 기타 

### 단순 선형 회귀 모형 
- 단순 선형 회귀 모델로 Y(결과)에 대한 X(정보)를 fitting한다 식은 아래와 같다
- 결과물은 model.coef (정보에 대한 가중치)와 model.intercept(절편)에 저장된다 
$$ Y \sim W_{1}*close + W_{2}*lag1+...+ W_{8}*lag7 + intercept $$ 



In [109]:
#basic linear regression 
model = linear_model.LinearRegression()
x_vars1 = ['close','lag1','lag2','lag3','lag4','lag5','lag6','lag7','volume']
model.fit(_622_data[x_vars1], _622_data["fu1"]) # model.fit(x,y)
print( model.coef_, model.intercept_)



[ 1.01392153e+00 -3.58383341e-02  5.35752551e-02 -2.34160124e-02
  5.23180416e-03 -1.71192837e-02  7.83890699e-03 -5.52170224e-03
 -6.88488726e-10] 0.11334864469257866


### 교호작용을 고려한 모델 
- 교호작용은 effect modification이라고 불리며, 한 변수에 의해 다른 변수의 효과가 변화하는 모델로 아래의 수식과 같음  
$$ Y \sim W_{1}*close + W_{2}*lag1+...+ W_{n}*(close * volume) + intercept $$ 


In [120]:
_622_data['clo_vol'] = _622_data['close']*_622_data['volume']
_622_data_eval['clo_vol'] = _622_data_eval['close']*_622_data_eval['volume']
_622_data_test['clo_vol'] = _622_data_test['close']*_622_data_test['volume']

/Users/seojiwan/anaconda3/envs/tensor_test/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/seojiwan/anaconda3/envs/tensor_test/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [121]:
model2 = linear_model.LinearRegression()
x_var2 = ['close','lag1','lag2','lag3','lag4','lag5','lag6','lag7','volume','clo_vol']
model2.fit(_622_data[x_var2], _622_data["fu1"]) # model.fit(x,y)
print( model2.coef_, model2.intercept_)

[ 1.01449625e+00 -3.59001565e-02  5.35686448e-02 -2.33310138e-02
  5.23686854e-03 -1.71603370e-02  7.83604171e-03 -5.56355044e-03
  8.14674744e-09 -2.63906396e-10] 0.09647488902456303


### 평가하기 
- 지수 예측: MAE, MSE 기반 비교 
- 방향 예측: 정밀도, 재현율 


In [122]:
_622_data[:3]

,date,open,high,low,close,volume,Name,lag1,lag2,lag3,lag4,lag5,lag6,lag7,fu1,fu7,fu30,clo_vol
7,2013-02-20,14.17,14.26,13.15,13.33,14725200,AAL,14.26,14.50,13.99,14.66,14.27,14.46,14.75,13.37,13.61,15.69,196286916.0
8,2013-02-21,13.62,13.95,12.90,13.37,11922100,AAL,13.33,14.26,14.50,13.99,14.66,14.27,14.46,13.57,13.90,15.72,159398477.0
9,2013-02-22,13.57,13.60,13.21,13.57,6071400,AAL,13.37,13.33,14.26,14.50,13.99,14.66,14.27,13.02,14.05,15.84,82388898.0


In [127]:
_622_data.iloc['close'][:2                       ]

TypeError: cannot do positional indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [close] of <class 'str'>